In [8]:
!pip install elasticsearch8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.8/477.8 kB 4.1 MB/s eta 0:00:00


In [1]:
import pandas as pd
import requests
from elasticsearch8 import Elasticsearch

In [6]:
INDEX_NAME = "building-permits"

es_client = Elasticsearch (
        'https://localhost:9200',
        verify_certs= False,
        ssl_show_warn= False,
        basic_auth=('elastic', 'elastic')
    )
es_client.indices.refresh(index=INDEX_NAME)

count = 0
es_data = []
try:
    while True:
        result = es_client.search(index=INDEX_NAME, body = {"from": count
                                                            ,'size' : 100,
        'query': {
            'match_all' : {}
        }
        }
        )
        for doc in result['hits']['hits']:
            es_data.append(doc['_source'])
        count += 100
except:
    print("Got all the Documents")

data = pd.DataFrame(es_data)

Got all the Documents


In [9]:
data

,council_ref,permit_number,issue_date,address,desc_of_works,estimated_cost_of_works,rbs_number,commence_by_date,completed_by_date,permit_certificate_type
109,BP-2012-1161,1183/2012/03008/1-12-000027,2012-08-08,76-98 Canning Street CARLTON VIC 3053,Stage 1: Bulk Excavation and Site Retention (O...,32500000,BS-U1183,2013-08-08,2015-08-08,Building Permit
142,BP-2012-1110,1183/2012/0299/1,2012-08-08,30-50 Elgin Street CARLTON VIC 3053,Living Carlton Stage 3 - Social (EN4). Bulk E...,32500000,BS-U1183,2013-08-08,2016-08-08,Building Permit
194,BP-2011-528/19,1162/110073/20,2012-07-13,Melbourne Park 200 Batman Avenue MELBOURNE VIC...,Stage 20: Part demolition of AAMI Park Stair. ...,99344207,BS-1162,2013-07-13,2014-04-01,Building Permit
206,BP-2011-593/6,27882/2011100/7,2012-07-04,786-798 Elizabeth Street MELBOURNE VIC 3000,Stage 7: Balance of works,163895000,BSU 27882,2013-07-04,2015-07-04,Building Permit
249,BP-2012-426/1,1215/20122312/2/3306,2012-07-02,31-49 Queens Bridge Street SOUTHBANK VIC 3006,Stage 2 - Structural detail podium. Columns an...,1500000,None,2013-07-02,2015-04-05,Building Permit
...,...,...,...,...,...,...,...,...,...,...
157975,BP-2012-426/2,1215/20122312/3,2012-07-31,31-49 Queens Bridge Street SOUTHBANK VIC 3006,Stage 3 - Variation to ground floor south boun...,600000,None,2012-07-31,2015-04-05,Building Permit
158575,BP-2008-907/1,1573/20130051/0,2009-06-23,Hotel Sophia 277-287 King Street MELBOURNE VIC...,Stage 2: To Completion. Roof top extension to ...,2000000,BS-1573,2014-03-13,2023-03-13,Building Permit
159662,BP-2006-5048/1,18514/005319/4,2006-07-13,CSL Limited 39-79 Poplar Road PARKVILLE VIC 3052,"Alterations to a building, building 5 Morgan L...",12500000,BS-U 18514,2007-04-13,2009-04-13,Building Permit
159758,BP-2006-4724/A,1092/981894/A,2006-07-26,27-29 Tattersalls Lane MELBOURNE VIC 3000,Construction of a new roof structure to a Bar/...,10000,BS-1092,2008-06-27,2009-06-27,Certificate of Final Inspection


In [10]:
def postcode(add):
    return add.split()[-1]

data['post_code'] = data['address'].apply(postcode)
postal_codes = data["post_code"].values.astype('str').tolist()
len(postal_codes)

63682

In [11]:
pip install pgeocode

In [12]:
import pgeocode
nomi = pgeocode.Nominatim('au')
au_post = nomi.query_postal_code(postal_codes)
post_count = au_post.groupby('postal_code').size()

In [13]:
au_post

,postal_code,country_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,3053,AU,"Carlton South, Carlton",Victoria,VIC,MELB NORTH WEST,NaN,NaN,NaN,-37.80055,144.96710,3.0
1,3053,AU,"Carlton South, Carlton",Victoria,VIC,MELB NORTH WEST,NaN,NaN,NaN,-37.80055,144.96710,3.0
2,3004,AU,"St Kilda Road Central, St Kilda Road Melbourne...",Victoria,VIC,MELBOURNE CITY,NaN,NaN,NaN,-37.82340,144.96760,3.0
3,3000,AU,Melbourne,Victoria,VIC,MELBOURNE CITY,NaN,NaN,NaN,-37.81300,144.96110,4.0
4,3006,AU,"South Wharf, Southbank",Victoria,VIC,MOORABBIN,NaN,NaN,NaN,-37.82675,144.95305,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
63677,3006,AU,"South Wharf, Southbank",Victoria,VIC,MOORABBIN,NaN,NaN,NaN,-37.82675,144.95305,4.0
63678,3000,AU,Melbourne,Victoria,VIC,MELBOURNE CITY,NaN,NaN,NaN,-37.81300,144.96110,4.0
63679,3052,AU,"Parkville, Melbourne University",Victoria,VIC,MELB NORTH WEST,NaN,NaN,NaN,-37.77245,144.94965,4.0
63680,3000,AU,Melbourne,Victoria,VIC,MELBOURNE CITY,NaN,NaN,NaN,-37.81300,144.96110,4.0


In [14]:
au_post['address'] = data['address'].values
au_post['issue_date'] = data['issue_date'].values
au_post['issue_date'] = pd.to_datetime(au_post['issue_date'])
au_post['year'] = au_post['issue_date'].dt.year

In [15]:
count_per_year_postal_code = au_post.groupby(['year', 'postal_code']).size().reset_index(name='count')
au_post = au_post.merge(count_per_year_postal_code, on=['year', 'postal_code'])

In [16]:
import plotly.express as px
selected_year = 2021
au_post_filtered = au_post[au_post['year'] == selected_year]

fig = px.scatter_mapbox(au_post_filtered, lat="latitude", lon="longitude",
                        color="postal_code", size='count', zoom=4, hover_name="count", hover_data=["postal_code", "count"],
                        color_continuous_scale=px.colors.cyclical.Twilight,
                        mapbox_style='open-street-map')
fig.show()

In [17]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [47]:
%%writefile app.py

import streamlit as st
import requests
import pgeocode
import pandas as pd
import plotly.express as px

def postcode(add):
    return add.split()[-1]

INDEX_NAME = "building-permits"

es_client = Elasticsearch (
        'https://localhost:9200',
        verify_certs= False,
        ssl_show_warn= False,
        basic_auth=('elastic', 'elastic')
    )
es_client.indices.refresh(index=INDEX_NAME)

count = 0
es_data = []
try:
    while True:
        result = es_client.search(index=INDEX_NAME, body = {"from": count
                                                            ,'size' : 100,
        'query': {
            'match_all' : {}
        }
        }
        )
        for doc in result['hits']['hits']:
            es_data.append(doc['_source'])
        count += 100
except:
    print("Got all the Documents")

data = pd.DataFrame(es_data)
data['post_code'] = data['address'].apply(postcode)
postal_codes = data["post_code"].values.astype('str').tolist()
nomi = pgeocode.Nominatim('au')
au_post = nomi.query_postal_code(postal_codes)
post_count = au_post.groupby('postal_code').size()

au_post['address'] = data['address'].values
au_post['issue_date'] = data['issue_date'].values

au_post['issue_date'] = pd.to_datetime(au_post['issue_date'])
au_post['year'] = au_post['issue_date'].dt.year

def plot_ind_years(data):
  count_per_year_postal_code = data.groupby(['year', 'postal_code']).size().reset_index(name='count')
  data = data.merge(count_per_year_postal_code, on=['year', 'postal_code'])
  return data

def plot_scatter_mapbox(data):
    fig = px.scatter_mapbox(data, lat="latitude", lon="longitude",
                            color="postal_code", size='count', zoom=4,
                            color_continuous_scale=px.colors.cyclical.Twilight,
                            mapbox_style='open-street-map')
    return fig

def plot_all_years(data):
  count_per_postal_code = data.groupby('postal_code').size().reset_index(name='count')
  data = data.merge(count_per_postal_code, on='postal_code')
  return data

st.title("Building Permits Scatter Map Plot by Year")

years = sorted(au_post['year'].unique())
selected_year = st.selectbox('Select Year', ['All Years'] + years)

if selected_year == 'All Years':
    fig = plot_all_years(au_post)
else:
    filtered_data = au_post[au_post['year'] == selected_year]
    fig = plot_ind_years(filtered_data)

out = plot_scatter_mapbox(fig)
st.plotly_chart(out)

Overwriting app.py


In [19]:
!pip install pyngrok

In [20]:
from pyngrok import ngrok

In [21]:
ngrok.set_auth_token("")

In [49]:
# Start Streamlit app
import os
os.system("streamlit run app.py &")

# Open a ngrok tunnel to the streamlit port
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at {public_url}")

Streamlit app is live at NgrokTunnel: "https://7930-34-23-17-117.ngrok-free.app" -> "http://localhost:8501"


In [48]:
ngrok.kill()

In [ ]:
from pyngrok import ngrok

tunnels = ngrok.get_tunnels()
tunnels

# try:
#     # Block until CTRL-C or some other terminating event
#     ngrok_process.proc.wait()
# except KeyboardInterrupt:
#     print(" Shutting down server.")

#     ngrok.kill()